In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np
import pymongo

In [2]:
# Read CSV into dataframe
shelterCSV = "Resources/aac_intakes_outcomes.csv"
shelter_df = pd.read_csv(shelterCSV)

In [3]:
# Read JSON into dataframe
# breedJSON = "Resources/breed_info_2.json"
with open('Resources/breed_info.json', 'r') as f:
    pets_attr_dict = json.load(f)
    
    
# Make a dictionary with just dog_breeds
dogs_attr_dict = pets_attr_dict["dog_breeds"]
dogs_attr_dict

# Make a dataframe from dog dictionary and reset the index
dogs_df = pd.DataFrame.from_dict(dogs_attr_dict)
dogs_df = dogs_df.transpose()
dogs_df = dogs_df.reset_index()
dogs_df.rename(columns={'index':'Breed'}, inplace=True)

# Rename Columns and sort by breed
dogs_df = dogs_df.rename(columns={" Adaptability":"Adaptability"," All Around Friendliness":"All Around Friendliness"})
dogs_df = dogs_df.sort_values('Breed', ascending=True)
dogs_df.head()

,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Potential For Playfulness,Potential For Weight Gain,Prey Drive,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential
0,Affenpinscher,3.0,3.0,4.0,2.0,3.0,5.0,5.0,1.0,4.0,...,4.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,2.0
1,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,4.0,1.0,5.0,5.0,4.0,2.0,2.0,5.0,5.0,5.0
2,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,5.0,4.0,5.0,3.0,3.0,4.0,2.0,3.0,3.0,4.0
3,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,5.0,4.0,4.0,5.0,4.0,5.0,1.0,5.0,2.0,4.0
4,Alaskan Klee Kai,3.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,2.0,...,4.0,2.0,5.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0


In [4]:
# Make List for Column Categories and Copy into dataframe
shelter_cols = ["breed","animal_type","age_upon_outcome_(days)","outcome_type","sex_upon_outcome","time_in_shelter_days"]
Shelter_df = shelter_df[shelter_cols].copy()

# Drop Data That Isn't Related to Dogs
Shelter_df = Shelter_df[Shelter_df.animal_type == 'Dog']


# Rename Columns
Shelter_df = Shelter_df.rename(columns={"age_upon_outcome_(days)":"outcome_age_days",
                                        "sex_upon_outcome":"gender"})

# Added more specific age of intake using exact 'time_in_shelter_days' 
Shelter_df['intake_age_days'] = Shelter_df['outcome_age_days'] - Shelter_df['time_in_shelter_days']

# Cleaning Up Breeds
Shelter_df["Breed"] = Shelter_df["breed"].str.split('/').str[0]
Shelter_df["Sub"] = Shelter_df["breed"].str.split('/').str[-1]

# Dropped the 'Mix' tag from breeds to match the dataframes better
Shelter_df["Breed"] = Shelter_df["Breed"].map(lambda x: x.rstrip(' Mix'))

# Dropped unwanted columns
Shelter_df = Shelter_df.drop(['animal_type', 'breed', 'Sub'], axis=1)

# Format number for outcome
Shelter_df = Shelter_df.astype({"outcome_age_days": float})
Shelter_df.head()

,outcome_age_days,outcome_type,gender,time_in_shelter_days,intake_age_days,Breed
0,3650.0,Return to Owner,Neutered Male,0.588194,3649.411806,Spinone Italiano
1,2555.0,Return to Owner,Neutered Male,1.259722,2553.740278,Spinone Italiano
2,2190.0,Return to Owner,Neutered Male,1.113889,2188.886111,Spinone Italiano
3,3650.0,Transfer,Neutered Male,4.970139,3645.029861,Dachshund
4,5840.0,Return to Owner,Neutered Male,0.119444,5839.880556,Shetland Sheepdog


In [5]:
# Sort on breed and get average times at shelter 
group_by_breed = Shelter_df.groupby('Breed')
time_in_shelter_days = group_by_breed['time_in_shelter_days'].mean()
outcome_age_days = group_by_breed['outcome_age_days'].mean()
intake_age_days = group_by_breed['intake_age_days'].mean()

# Make dataframes for the average calculations
time_in_df = pd.DataFrame(time_in_shelter_days)
outcome_age_df = pd.DataFrame(outcome_age_days)
intake_age_df = pd.DataFrame(intake_age_days)

# Concatenate all dataframe averages into single dataframe
shelter_details_df=pd.concat([time_in_df,outcome_age_df,intake_age_df], axis=1)
shelter_details_df.head()

,time_in_shelter_days,outcome_age_days,intake_age_days
Breed,,,
Affenpinscher,2.334028,886.428571,884.094544
Afghan Hound,23.982639,730.000000,706.017361
Airedale Terrier,12.819610,1039.523810,1026.704200
Akbash,22.605972,584.000000,561.394028
Akita,13.745076,1290.984848,1277.239773


In [6]:
# Combined shelter averages with dog breed data, drop unmatching entries
all_dog_data_df = pd.merge(dogs_df, shelter_details_df, on=["Breed"], how='outer')
all_dog_data_df = all_dog_data_df.dropna(how='any')
all_dog_data_df

,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential,time_in_shelter_days,outcome_age_days,intake_age_days
0,Affenpinscher,3.0,3.0,4.0,2.0,3.0,5.0,5.0,1.0,4.0,...,3.0,1.0,2.0,1.0,3.0,3.0,2.0,2.334028,886.428571,884.094544
1,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,5.0,4.0,2.0,2.0,5.0,5.0,5.0,23.982639,730.000000,706.017361
2,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,3.0,3.0,4.0,2.0,3.0,3.0,4.0,12.819610,1039.523810,1026.704200
3,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,5.0,4.0,5.0,1.0,5.0,2.0,4.0,13.745076,1290.984848,1277.239773
5,Alaskan Malamute,2.0,4.0,5.0,3.0,4.0,1.0,4.0,5.0,2.0,...,4.0,4.0,5.0,1.0,5.0,2.0,5.0,4.492851,1221.764706,1217.271855
6,American Bulldog,2.0,3.0,4.0,3.0,3.0,1.0,5.0,1.0,2.0,...,4.0,4.0,2.0,1.0,2.0,2.0,3.0,39.187939,982.052493,942.864554
9,American Foxhound,2.0,5.0,5.0,4.0,4.0,1.0,4.0,3.0,5.0,...,2.0,3.0,5.0,1.0,4.0,4.0,5.0,8.508907,711.086957,702.578050
10,American Pit Bull Terrier,3.0,4.0,5.0,3.0,4.0,3.0,5.0,4.0,1.0,...,4.0,3.0,3.0,1.0,2.0,3.0,4.0,30.428940,802.736842,772.307902
11,American Staffordshire Terrier,2.0,4.0,4.0,3.0,4.0,2.0,4.0,2.0,2.0,...,3.0,3.0,4.0,2.0,2.0,3.0,3.0,31.827047,1019.943609,988.116562
15,Australian Cattle Dog,3.0,4.0,5.0,3.0,4.0,2.0,5.0,3.0,4.0,...,4.0,3.0,2.0,3.0,4.0,4.0,4.0,19.901094,883.881415,863.980321


In [7]:
# initialize percentage list and list out breeds
all_breeds = list(all_dog_data_df["Breed"])
all_percentages = []

# Loop breed list and determine how often dogs get a good outcome (being adopted or returned to owner)
for breed in all_breeds:
    breed_df = Shelter_df.loc[Shelter_df['Breed'] == breed] ##set dataframe of breeds
    number_adopted = (breed_df['outcome_type'] == "Adoption").sum() ##sum all outcomes in 'Adoption'
    returned_to_owner = (breed_df['outcome_type'] == "Return to Owner").sum() ##sum all outcomes: 'Return to Owner'
    total_outcomes = breed_df['outcome_type'].count() ##count every outcome
    total_good_outcomes = number_adopted+returned_to_owner ##total desireable outcomes
    pct_good_outcomes = total_good_outcomes/total_outcomes ##get dersireable outcome percentage
    pct_good_outcomes = "{0:.0%}".format(pct_good_outcomes) ##format percentage
    all_percentages.append(pct_good_outcomes) ##append outcome percentages to list

    
# Make dict from Breed and Percentage data
good_outcome_pct_dict = {"Breed":all_breeds,"Good Outcome Percentage":all_percentages}

# Transform dict into dataframe
good_outcome_df = pd.DataFrame(good_outcome_pct_dict)
good_outcome_df

,Breed,Good Outcome Percentage
0,Affenpinscher,100%
1,Afghan Hound,100%
2,Airedale Terrier,81%
3,Akita,79%
4,Alaskan Malamute,88%
5,American Bulldog,76%
6,American Foxhound,74%
7,American Pit Bull Terrier,72%
8,American Staffordshire Terrier,74%
9,Australian Cattle Dog,78%


In [8]:
# Merge dog and good outcome dataframes, rename columns
final_dog_data_df = pd.merge(all_dog_data_df, good_outcome_df)
final_dog_data_df = final_dog_data_df.rename(columns={"time_in_shelter_days":"Avg Days In Shelter",
                                                  "outcome_age_days":"Avg Age Upon Outcome",
                                                  "intake_age_days":"Avg Age On Intake"})
final_dog_data_df

,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential,Avg Days In Shelter,Avg Age Upon Outcome,Avg Age On Intake,Good Outcome Percentage
0,Affenpinscher,3.0,3.0,4.0,2.0,3.0,5.0,5.0,1.0,4.0,...,1.0,2.0,1.0,3.0,3.0,2.0,2.334028,886.428571,884.094544,100%
1,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,4.0,2.0,2.0,5.0,5.0,5.0,23.982639,730.000000,706.017361,100%
2,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,3.0,4.0,2.0,3.0,3.0,4.0,12.819610,1039.523810,1026.704200,81%
3,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,4.0,5.0,1.0,5.0,2.0,4.0,13.745076,1290.984848,1277.239773,79%
4,Alaskan Malamute,2.0,4.0,5.0,3.0,4.0,1.0,4.0,5.0,2.0,...,4.0,5.0,1.0,5.0,2.0,5.0,4.492851,1221.764706,1217.271855,88%
5,American Bulldog,2.0,3.0,4.0,3.0,3.0,1.0,5.0,1.0,2.0,...,4.0,2.0,1.0,2.0,2.0,3.0,39.187939,982.052493,942.864554,76%
6,American Foxhound,2.0,5.0,5.0,4.0,4.0,1.0,4.0,3.0,5.0,...,3.0,5.0,1.0,4.0,4.0,5.0,8.508907,711.086957,702.578050,74%
7,American Pit Bull Terrier,3.0,4.0,5.0,3.0,4.0,3.0,5.0,4.0,1.0,...,3.0,3.0,1.0,2.0,3.0,4.0,30.428940,802.736842,772.307902,72%
8,American Staffordshire Terrier,2.0,4.0,4.0,3.0,4.0,2.0,4.0,2.0,2.0,...,3.0,4.0,2.0,2.0,3.0,3.0,31.827047,1019.943609,988.116562,74%
9,Australian Cattle Dog,3.0,4.0,5.0,3.0,4.0,2.0,5.0,3.0,4.0,...,3.0,2.0,3.0,4.0,4.0,4.0,19.901094,883.881415,863.980321,78%


In [9]:
# Set breed as index for dog data, convert to json
final_dog_data_df = final_dog_data_df.set_index(keys="Breed")
final_data_json = final_dog_data_df.to_json(orient='index')
print(final_data_json)

{"Affenpinscher":{"Adaptability":3.0,"All Around Friendliness":3.0," Exercise Needs":4.0," Health Grooming":2.0," Trainability":3.0,"Adapts Well to Apartment Living":5.0,"Affectionate with Family":5.0,"Amount Of Shedding":1.0,"Dog Friendly":4.0,"Drooling Potential":1.0,"Easy To Groom":3.0,"Easy To Train":2.0,"Energy Level":4.0,"Exercise Needs":3.0,"Friendly Toward Strangers":3.0,"General Health":4.0,"Good For Novice Owners":4.0,"Incredibly Kid Friendly Dogs":1.0,"Intelligence":4.0,"Intensity":3.0,"Potential For Mouthiness":4.0,"Potential For Playfulness":4.0,"Potential For Weight Gain":3.0,"Prey Drive":3.0,"Sensitivity Level":3.0,"Size":1.0,"Tendency To Bark Or Howl":2.0,"Tolerates Being Alone":1.0,"Tolerates Cold Weather":3.0,"Tolerates Hot Weather":3.0,"Wanderlust Potential":2.0,"Avg Days In Shelter":2.3340277776,"Avg Age Upon Outcome":886.4285714286,"Avg Age On Intake":884.094543651,"Good Outcome Percentage":"100%"},"Afghan Hound":{"Adaptability":4.0,"All Around Friendliness":4.0," 

In [10]:
# Merge shelter details with good outcomes, set index as Breed, convert dataframe to json
shelter_details_df = shelter_details_df.reset_index(drop=False)
shelter_details_df = pd.merge(shelter_details_df, good_outcome_df)
shelter_details_df = shelter_details_df.set_index(keys="Breed")
final_shelter_json = shelter_details_df.to_json(orient='index')
print(final_shelter_json)

{"Affenpinscher":{"time_in_shelter_days":2.3340277776,"outcome_age_days":886.4285714286,"intake_age_days":884.094543651,"Good Outcome Percentage":"100%"},"Afghan Hound":{"time_in_shelter_days":23.982638891,"outcome_age_days":730.0,"intake_age_days":706.017361109,"Good Outcome Percentage":"100%"},"Airedale Terrier":{"time_in_shelter_days":12.8196097881,"outcome_age_days":1039.5238095238,"intake_age_days":1026.7041997357,"Good Outcome Percentage":"81%"},"Akita":{"time_in_shelter_days":13.745075758,"outcome_age_days":1290.9848484848,"intake_age_days":1277.2397727268,"Good Outcome Percentage":"79%"},"Alaskan Malamute":{"time_in_shelter_days":4.492851307,"outcome_age_days":1221.7647058824,"intake_age_days":1217.2718545753,"Good Outcome Percentage":"88%"},"American Bulldog":{"time_in_shelter_days":39.1879392678,"outcome_age_days":982.0524934383,"intake_age_days":942.8645541705,"Good Outcome Percentage":"76%"},"American Foxhound":{"time_in_shelter_days":8.508907004,"outcome_age_days":711.0869

In [11]:
# Set index of dog dataframe to Breed and then convert to Json
dogs_df = dogs_df.set_index(keys="Breed")
final_dog_json = dogs_df.to_json(orient='index')
print(final_dog_json)

{"Affenpinscher":{"Adaptability":3.0,"All Around Friendliness":3.0," Exercise Needs":4.0," Health Grooming":2.0," Trainability":3.0,"Adapts Well to Apartment Living":5.0,"Affectionate with Family":5.0,"Amount Of Shedding":1.0,"Dog Friendly":4.0,"Drooling Potential":1.0,"Easy To Groom":3.0,"Easy To Train":2.0,"Energy Level":4.0,"Exercise Needs":3.0,"Friendly Toward Strangers":3.0,"General Health":4.0,"Good For Novice Owners":4.0,"Incredibly Kid Friendly Dogs":1.0,"Intelligence":4.0,"Intensity":3.0,"Potential For Mouthiness":4.0,"Potential For Playfulness":4.0,"Potential For Weight Gain":3.0,"Prey Drive":3.0,"Sensitivity Level":3.0,"Size":1.0,"Tendency To Bark Or Howl":2.0,"Tolerates Being Alone":1.0,"Tolerates Cold Weather":3.0,"Tolerates Hot Weather":3.0,"Wanderlust Potential":2.0},"Afghan Hound":{"Adaptability":4.0,"All Around Friendliness":4.0," Exercise Needs":4.0," Health Grooming":2.0," Trainability":3.0,"Adapts Well to Apartment Living":5.0,"Affectionate with Family":5.0,"Amount 

In [12]:
# Setup mongo client to connect to localhost and create database "ETL_project_dogs"
mongo_db = pymongo.MongoClient('localhost', 27017)['ETL_project_dogs']

# Create table in mongo by loading in json data, clear possible entries in table before load
db_dogs = mongo_db['dog_data' ] #table name
db_dogs.remove() #clear table
db_dogs.insert(json.loads(final_dog_json)) #data being loaded

db_dogs = mongo_db['shelter_data' ]
db_dogs.remove()
db_dogs.insert(json.loads(final_shelter_json))

db_dogs = mongo_db['total_data' ]
db_dogs.remove()
db_dogs.insert(json.loads(final_data_json))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: insert is deprecated. Use insert

ObjectId('5cb38a39ac330246ec451bb8')

In [211]:
# consolidating dog breed categories to present easier to view table
final_dog_data_df['Friendly'] =((final_dog_data_df['All Around Friendliness']+
                               final_dog_data_df['Affectionate with Family']+
                               final_dog_data_df['Dog Friendly']+
                               final_dog_data_df['Friendly Toward Strangers']+
                               final_dog_data_df['Incredibly Kid Friendly Dogs'] +
                               final_dog_data_df['Potential For Playfulness'])/6).astype('int')
final_dog_data_df['Intelligence'] =((final_dog_data_df['Adaptability']+
                               final_dog_data_df['Affectionate with Family']+
                               final_dog_data_df[' Trainability']+
 #                              final_dog_data_df['Trainability']+
                               final_dog_data_df['Easy To Train']+
                               final_dog_data_df['Intelligence'])/4).astype('int')
final_dog_data_df['Tolerance'] =((final_dog_data_df['Easy To Groom']+
                               final_dog_data_df['Tolerates Being Alone']+
                               final_dog_data_df['Tolerates Cold Weather']+
                               final_dog_data_df['Tolerates Hot Weather'])/4).astype('int')
final_dog_data_df['Negative_Attributes'] =((final_dog_data_df['Amount Of Shedding']+
                               final_dog_data_df['Affectionate with Family']+
                               final_dog_data_df['Drooling Potential']+
                               final_dog_data_df['Prey Drive']+
                               final_dog_data_df['Sensitivity Level']+
                               final_dog_data_df['Tendency To Bark Or Howl']+
                               final_dog_data_df['Wanderlust Potential']+
                               final_dog_data_df['Potential For Mouthiness']+
                               final_dog_data_df['Potential For Weight Gain'])/9).astype('int')
dog_attributes_df=pd.DataFrame({
                                           "Breed":final_dog_data_df["Breed"],
                                           "Friendly":final_dog_data_df["Friendly"],
                                           "Intelligence":final_dog_data_df["Intelligence"],
                                           "Tolerance":final_dog_data_df["Tolerance"],
                                           "Negative Attributes":final_dog_data_df["Negative_Attributes"],
                                           "Apartment Living":final_dog_data_df["Adapts Well to Apartment Living"],
                                           "Energy Level":final_dog_data_df["Energy Level"],
                                           "Exercise Needs":final_dog_data_df["Exercise Needs"],
                                           "General Health":final_dog_data_df["General Health"],
                                           "Intensity":final_dog_data_df["Intensity"],
                                           "Size":final_dog_data_df["Size"],
                                           "Good For Novice Owners":final_dog_data_df["Good For Novice Owners"],
                                           "Avg Days In Shelter":final_dog_data_df["Avg Days In Shelter"],
                                           "Avg Age Upon Outcome":final_dog_data_df["Avg Age Upon Outcome"],
                                           "Avg Age On Intake":final_dog_data_df["Avg Age On Intake"],
                                           "Good Outcome%":final_dog_data_df["Good Outcome Percentage"],
                                                 })

dog_attributes_df.head()

,Breed,Friendly,Intelligence,Tolerance,Negative Attributes,Apartment Living,Energy Level,Exercise Needs,General Health,Intensity,Size,Good For Novice Owners,Avg Days In Shelter,Avg Age Upon Outcome,Avg Age On Intake,Good Outcome%
0,Afghan Hound,4,4,3,3,5.0,5.0,4.0,3.0,2.0,4.0,3.0,46.824306,730.000000,683.175694,100%
1,Airedale Terrier,4,5,2,3,1.0,5.0,5.0,3.0,3.0,3.0,2.0,12.259375,1039.166667,1026.907292,67%
2,Akita,2,4,2,4,2.0,4.0,4.0,4.0,3.0,4.0,2.0,13.203980,1357.692308,1344.488328,69%
3,Alaskan Malamute,3,4,2,3,1.0,5.0,5.0,4.0,5.0,4.0,1.0,6.107824,1161.666667,1155.558843,80%
4,American Bulldog,3,4,2,3,1.0,4.0,4.0,3.0,4.0,4.0,1.0,31.551841,979.920635,948.368794,83%


In [212]:
dog_attributes_df = dog_attributes_df.set_index(keys="Breed")
condensed_dog_json = dog_attributes_df.to_json(orient='index')
print(condensed_dog_json)

{"Afghan Hound":{"Friendly":4,"Intelligence":4,"Tolerance":3,"Negative Attributes":3,"Apartment Living":5.0,"Energy Level":5.0,"Exercise Needs":4.0,"General Health":3.0,"Intensity":2.0,"Size":4.0,"Good For Novice Owners":3.0,"Avg Days In Shelter":46.82430556,"Avg Age Upon Outcome":730.0,"Avg Age On Intake":683.17569444,"Good Outcome%":"100%"},"Airedale Terrier":{"Friendly":4,"Intelligence":5,"Tolerance":2,"Negative Attributes":3,"Apartment Living":1.0,"Energy Level":5.0,"Exercise Needs":5.0,"General Health":3.0,"Intensity":3.0,"Size":3.0,"Good For Novice Owners":2.0,"Avg Days In Shelter":12.2593750005,"Avg Age Upon Outcome":1039.1666666667,"Avg Age On Intake":1026.9072916662,"Good Outcome%":"67%"},"Akita":{"Friendly":2,"Intelligence":4,"Tolerance":2,"Negative Attributes":4,"Apartment Living":2.0,"Energy Level":4.0,"Exercise Needs":4.0,"General Health":4.0,"Intensity":3.0,"Size":4.0,"Good For Novice Owners":2.0,"Avg Days In Shelter":13.2039797015,"Avg Age Upon Outcome":1357.6923076923,"

In [213]:
db_dogs = mongo_db['total_data_condensed' ]
db_dogs.remove()
db_dogs.insert(json.loads(condensed_dog_json))

C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


ObjectId('5cb3bec25226d55934cfb9b5')